# Join Transaktionen und Lines

Ein Left-Join von Transaktionen und Lines ergibt eine Tabelle, die 13 Zeilen mehr als die Lines-Tabelle hat. Das liegt daran, dass es 13 Transaktionen gibt, die keine zugehörigen Lines haben. 

Die Daten sind aber konsistent, da die entsprechenden Transaktionen n_lines = 0 haben.

Von den 13 Transaktionen ist eine gelabelt. 

Fazit: Da es sich nur um 13 Transaktionen handelt und nur eine davon gelabelt ist, können sie entfernt werden.

In [2]:
import polars as pl
from pathlib import Path

data_dir = Path("../data")

transactions = pl.scan_parquet(data_dir / "transactions_train_3.parquet")
lines = pl.scan_parquet(data_dir / "transaction_lines_train_3.parquet")

joined = transactions.join(
  lines,
  left_on="id",
  right_on="transaction_id",
  suffix="__lines",
  how="left",
)

In [3]:
n_rows_lines = lines.select(pl.len()).collect().item()
n_rows_lines = f"{n_rows_lines:_}"
print(f"{'Number of rows in lines':<55}{n_rows_lines:>15}")

n_rows_joined = joined.select(pl.len()).collect().item()
n_rows_joined = f"{n_rows_joined:_}"
print(f"{'Number of rows in joined transactions and lines':<55}{n_rows_joined:>15}")

print(f"Difference in number of rows: {int(n_rows_joined) - int(n_rows_lines)}")

Number of rows in lines                                     15_793_671
Number of rows in joined transactions and lines             15_793_684
Difference in number of rows: 13


Die Join-Tabelle aus Transaktionen und Lines hat 13 Zeilen mehr als die Lines-Tabelle.

Grund: 13 Transaktionen haben keine matching lines

In [8]:
joined.select(["id", "id__lines"]).describe().filter(pl.col("statistic").is_in(["count", "null_count"]))

statistic,id,id__lines
str,str,f64
"""count""","""15793684""",1.5793671e7
"""null_count""","""0""",13.0


Die Daten sind konsistent: bei fehlenden id_lines ist die Spalte n_lines = 0

In [9]:
joined.select(["id", "id__lines", "n_lines", "damage"]).filter(pl.col("id__lines").is_null()).describe().filter(pl.col("statistic").is_in(["count", "null_count", "mean"]))

statistic,id,id__lines,n_lines,damage
str,str,f64,f64,f64
"""count""","""13""",0.0,13.0,1.0
"""null_count""","""0""",13.0,0.0,12.0
"""mean""",null,null,0.0,7.92


Es gibt allerdings auch Transaktionen mit n_lines = 0, denen Lines zugeordnet sind (allerdings ist dann was_voided = true, somit ist die Konsistenz erhalten). 

In [15]:
n_transactions_with_zero_lines =  joined.select("id", "n_lines", "id__lines", "damage").filter(pl.col("n_lines") == 0).select("id").unique().select(pl.len()).collect().item()
print(f"Number of transactions with zero lines: {n_transactions_with_zero_lines:_}")

joined.select("id", "n_lines", "id__lines", "was_voided", "damage").filter(pl.col("n_lines") == 0).describe()

Number of transactions with zero lines: 104


statistic,id,n_lines,id__lines,was_voided,damage
str,str,f64,f64,f64,f64
"""count""","""120""",120.0,107.0,107.0,15.0
"""null_count""","""0""",0.0,13.0,13.0,105.0
"""mean""",null,0.0,1.149533,1.0,8.778667
"""std""",null,0.0,0.358291,null,6.722329
"""min""","""00cf6219-d731-4a1f-9cd1-bbd556…",0.0,1.0,1.0,1.17
"""25%""",null,0.0,1.0,null,3.92
"""50%""",null,0.0,1.0,null,6.38
"""75%""",null,0.0,1.0,null,14.32
"""max""","""fe0eb499-d2e6-4ea8-9ade-94ee35…",0.0,2.0,1.0,20.79


12 der Transaktionen mit fehlenden Lines sind nicht gelabelt bei einer handelt es sich um Betrug

In [3]:
(joined
  .filter(pl.col("id__lines").is_null())
  .group_by("label", "n_lines")
  .agg(pl.len().alias("count"))
).collect()

label,n_lines,count
str,i64,u32
"""FRAUD""",0,1
"""UNKNOWN""",0,12
